### Challenge 16: CBC bitflipping attacks

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [10]:
import cryptopals as cp
from numpy.random import randint

<div class="alert alert-block alert-info">
Generate a random AES key.
</div>

In [9]:
# ---------UNKNOWN PARAMETERS ------------
unknown_key = bytes(list(randint(0, 256, 16)))
# ---------END UNKNOWN PARAMETERS---------

<div class="alert alert-block alert-info">
    
Combine your padding code and CBC code to write two functions.

The first function should take an arbitrary input string, prepend the string:

```"comment1=cooking%20MCs;userdata="```

.. and append the string:

```";comment2=%20like%20a%20pound%20of%20bacon"```

The function should quote out the ";" and "=" characters.

The function should then pad out the input to the 16-byte AES block length and encrypt it under the random AES key.
</div>

In [15]:
def Challenge16Part1(data, key):

    prepend_str = b'comment1=cooking%20MCs;userdata='
    append_str = b';comment2=%20like%20a%20pound%20of%20bacon'

    data = data.replace(b';', b'\";\"')
    data = data.replace(b'=', b'\"=\"')

    plaintext = prepend_str + data + append_str
    plaintext = cp.PKCS7_pad(plaintext)

    ciphertext = cp.AESEncrypt(plaintext, key, mode='CBC')

    return(ciphertext)

<div class="alert alert-block alert-info">
    
The second function should decrypt the string and look for the characters ";admin=true;" (or, equivalently, decrypt, split the string on ";", convert each resulting string into 2-tuples, and look for the "admin" tuple).

Return true or false based on whether the string exists.
</div>

In [16]:
def Challenge16Part2(ciphertext, key):

    plaintext = cp.AESDecrypt(ciphertext, key, mode='CBC')
    plaintext = cp.strip_PKCS7_pad(plaintext)

    if plaintext.find(b';admin=true;') >= 0:
        return(True)
    else:
        return(False)

<div class="alert alert-block alert-info">
If you've written the first function properly, it should not be possible to provide user input to it that will generate the string the second function is looking for. We'll have to break the crypto to do that.
</div>

In [19]:
# %% Try the trivial solution:

payload = b';admin=true'
ct = Challenge16Part1(payload, unknown_key)

# Check to see if we were successful...

if Challenge16Part2(ct, unknown_key) is True:
    print('*** Success')
else:
    print('*** Try again loser!!!')

*** Try again loser!!!


<div class="alert alert-block alert-info">
    
Instead, modify the ciphertext (without knowledge of the AES key) to accomplish this.

You're relying on the fact that in CBC mode, a 1-bit error in a ciphertext block:
- Completely scrambles the block the error occurs in
- Produces the identical 1-bit error(/edit) in the next ciphertext block.

    <div class="alert alert-block alert-warning">
        
    ### Stop and think for a second.
    
    Before you implement this attack, answer this question: why does CBC mode have this property?
        
    </div>
    
</div>

---
Ok, as expected, the easy way didn't work.  Let's find out where our payload starts changing the ciphertext, and then come up with a payload that lets us fully control one of the AES ciphertext blocks.

In [25]:
payload = b''
last_ct = Challenge16Part1(payload, unknown_key)

for payload_len in range(1, 20):
    payload = b'\x00' * payload_len
    new_ct = Challenge16Part1(payload, unknown_key)
    if new_ct[32:48] == last_ct[32:48]:
        print(f"Found edge at: {payload_len}")
    last_ct = new_ct

Found edge at: 17
Found edge at: 18
Found edge at: 19


---
So...if we make a payload of 17 bytes long, we can completely control Block 2 (base 0) of the ciphertext.  That means I can recover the IV bytes used to encrypt that block by XOR'ing together my all b'\x00' payload with the ciphertext.  In otherwords, the key stream for that block IS the ciphertext!

In [43]:
payload = b'\x00' * 16
ct = Challenge16Part1(payload, unknown_key)
block_2_keystream = ct[16:32]

# Now just replace the block 1 (base 0) ciphertext with our chosen plaintext,
# XORed with the block 2 keystream

chosen_pt = b';admin=true;    '
chosen_ct = cp.bitwise_xor(chosen_pt, block_2_keystream)
ct = ct[:16] + chosen_ct + ct[32:]

# %%
# Check to see if we were successful...

if Challenge16Part2(ct, unknown_key) is True:
    print('\n*** Admin rights granted.\n')
else:
    print('Try again loser')

# %%
# Look inside... this is what the modified plaintext looks like.
pt = cp.AESDecrypt(ct, unknown_key, mode='CBC')
print(cp.strip_PKCS7_pad(pt))

### What did we learn?

Back to the two properties of AES-CBC related to 1-bit "errors":
    
>You're relying on the fact that in CBC mode, a 1-bit error in a ciphertext block:

>- Completely scrambles the block the error occurs in
>- Produces the identical 1-bit error(/edit) in the next ciphertext block.

By supplying all '0' plaintext for a block, we were able to learn the portion of the AES IV output corresponding to a targeted block.  Once that's known, it's possible to control the plaintext in that block by manipulating the ciphertext of the prior block.

![CBC Decryption - Annotated from Wikipedia](CBCDecrypt.png)

In the image above, we're modifying CT Block 1.  This allows us to "edit" the plaintext in PT Block 2 (note:  CT Block 1's output is scrambled since our edit causes the AES output to change for that block). 

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)